In [16]:
import pandas as pd
import numpy as np
import warnings
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from tensorflow.python.keras import callbacks
from keras import backend as K
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from keras.layers import Dense, Dropout
from sklearn.model_selection import KFold
from keras.optimizers import Adam, SGD
from keras.callbacks import EarlyStopping

warnings.filterwarnings(action='ignore')

In [26]:
df = pd.read_csv("modified_train_0417.csv")
x_train = df.drop(columns=['loan_status', 'addr_state', 'loan_amnt_intallment_rate'])
y_train = df['loan_status']

In [27]:
nan_indices = np.isnan(x_train).any(axis=1)
x_train = x_train[~nan_indices]
y_train = y_train[~nan_indices]

In [28]:
x_train.shape

(1091767,)

In [29]:
# minmax scaler 찾아보기
scaler = StandardScaler()
X_scaled = scaler.fit_transform(x_train)

# PCA로 차원 축소
pca = PCA(n_components=90)
X_pca = pca.fit_transform(X_scaled)
x_train = X_pca.astype('float32')
# stratify로 학습에 용이하게 비율 유지
x_train, x_validation, y_train, y_validation = train_test_split(x_train, y_train, test_size=0.2, stratify=y_train)

In [30]:
y_train=to_categorical(y_train, 2).astype(int)
y_validation=to_categorical(y_validation, 2).astype(int)
# from sklearn.preprocessing import OneHotEncoder
# encoder = OneHotEncoder()
# y_train = y_train.values
# y_train = encoder.fit_transform(y_train.reshape(-1, 1))

In [31]:
def f1_score(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())    
    f1_val = 2 * (precision * recall) / (precision + recall + K.epsilon())
    return f1_val

def recall(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

In [20]:
def build_model(optimizer):
    # Define and compile the model
    model = keras.Sequential([
        Dense(45, input_dim=90, activation='relu'),
        Dropout(0.2),
        Dense(45, activation='relu'),
        Dropout(0.2),
        Dense(45, activation='relu'),
        Dropout(0.2),
        Dense(10),
        Dense(2, activation='softmax')
    ])
    model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])
    return model
skfold = KFold(random_state=30,
           n_splits=5,
           shuffle=True
          )
model = KerasClassifier(build_fn = build_model)
parameters = {'batch_size': [1000,
                             10000],
              'epochs': [10,
                         20,
                         40,
                         60,
                         80,
                         100],
              'optimizer': ['adam', 
                            'SGD', 
                            'rmsprop'
                            ]}
grid_search = GridSearchCV(estimator = model,
                           param_grid = parameters,
                           scoring = 'f1',
                           cv = skfold)
# early_stopping = EarlyStopping(monitor='val_loss',min_delta=0.01)
grid_search.fit(x_train, y_train)
print("최고의 파라미터 :", grid_search.best_params_)
print("최고 평균 정확도 : {}".format(grid_search.best_score_))

Epoch 1/10
874/874 [==============================] - 2s 2ms/step - loss: 0.4714 - accuracy: 0.7964
Epoch 2/10
874/874 [==============================] - 2s 2ms/step - loss: 0.4601 - accuracy: 0.7983
Epoch 3/10
874/874 [==============================] - 2s 2ms/step - loss: 0.4583 - accuracy: 0.7989
Epoch 4/10
874/874 [==============================] - 2s 2ms/step - loss: 0.4571 - accuracy: 0.7991
Epoch 5/10
874/874 [==============================] - 2s 2ms/step - loss: 0.4565 - accuracy: 0.7993
Epoch 6/10
874/874 [==============================] - 2s 2ms/step - loss: 0.4559 - accuracy: 0.7997
Epoch 7/10
874/874 [==============================] - 2s 2ms/step - loss: 0.4553 - accuracy: 0.7999
Epoch 8/10
874/874 [==============================] - 2s 2ms/step - loss: 0.4550 - accuracy: 0.8001
Epoch 9/10
874/874 [==============================] - 2s 2ms/step - loss: 0.4546 - accuracy: 0.8000
Epoch 10/10
6824/6824 [==============================] - 1s 196us/step
Epoch 1/10
874/874 [=========

In [37]:
# 최고의 파라미터 : {'batch_size': 1000, 'epochs': 10, 'optimizer': 'adam'}
valid_accs, valid_f1s, valid_recalls, valid_precisions = [], [], [], []

# layers
model = keras.Sequential([
    Dense(45, input_dim=90, activation='relu'),
    Dropout(0.2),
    Dense(45, activation='relu'),
    Dropout(0.2),
    Dense(45, activation='relu'),
    Dropout(0.2),
    Dense(10),
    Dense(2, activation='softmax')
])
# optimizer
optimizer = Adam(learning_rate=0.01)
# compile
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy", f1_score, recall, precision])
skf = KFold(n_splits=5, shuffle=True, random_state=30)
for train_index, val_index in skf.split(x_train, y_train):
    X_train_fold, X_val_fold = x_train[train_index], x_train[val_index]
    Y_train_fold, Y_val_fold = y_train[train_index], y_train[val_index]

    # Train the model
    model.fit(X_train_fold, Y_train_fold, 
              batch_size=1000, 
              epochs=10, 
              verbose=1, 
              #callbacks=[EarlyStopping(min_delta=0.001, monitor="loss")]
              )

    # Evaluate the model on validation data
    valid_loss, valid_acc, valid_f1, valid_recall, valid_precision = model.evaluate(X_val_fold, Y_val_fold)
    valid_accs.append(valid_acc)
    valid_f1s.append(valid_f1)
    valid_recalls.append(valid_recall)
    valid_precisions.append(valid_precision)
    print("===================================")
    print("Validation accuracy:", valid_acc)
    print("Validation F1-score:", valid_f1)
    print("Validation recall:", valid_recall)
    print("Validation precision:", valid_precision)
print("###################################")
print("===================================")
print("avg Validation accuracy:", np.mean(valid_accs))
print("avg Validation F1-score:", np.mean(valid_f1s))
print("avg Validation recall:", np.mean(valid_recalls))
print("avg Validation precision:", np.mean(valid_precisions))


Epoch 1/10
699/699 [==============================] - 2s 3ms/step - loss: 0.4662 - accuracy: 0.7967 - f1_score: 0.7967 - recall: 0.7967 - precision: 0.7967
Epoch 2/10
699/699 [==============================] - 2s 3ms/step - loss: 0.4598 - accuracy: 0.7982 - f1_score: 0.7982 - recall: 0.7982 - precision: 0.7982
Epoch 3/10
699/699 [==============================] - 2s 4ms/step - loss: 0.4585 - accuracy: 0.7986 - f1_score: 0.7986 - recall: 0.7986 - precision: 0.7986
Epoch 4/10
699/699 [==============================] - 3s 4ms/step - loss: 0.4579 - accuracy: 0.7986 - f1_score: 0.7986 - recall: 0.7986 - precision: 0.7986
Epoch 5/10
699/699 [==============================] - 2s 3ms/step - loss: 0.4576 - accuracy: 0.7985 - f1_score: 0.7985 - recall: 0.7985 - precision: 0.7985
Epoch 6/10
699/699 [==============================] - 2s 3ms/step - loss: 0.4573 - accuracy: 0.7989 - f1_score: 0.7989 - recall: 0.7989 - precision: 0.7989
Epoch 7/10
699/699 [==============================] - 2s 3ms/ste

In [41]:
m_df = df.drop(columns=['addr_state', 'Unnamed: 0', 'loan_amnt_intallment_rate'])
m_df = m_df.corr('spearman')
m_df.to_csv('corr_0417.csv')

In [45]:
corr_df = pd.read_csv('corr_0417.csv')
corr_df

,Unnamed: 0,loan_amnt,term,int_rate,installment,sub_grade,emp_length,annual_inc,loan_status,dti,...,medical,moving,other,renewable_energy,small_business,vacation,wedding,Not Verified,Source Verified,Verified
0,loan_amnt,1.000000,0.419421,0.107936,0.968238,0.110776,0.054063,0.467790,0.079226,0.050908,...,-0.076840,-0.070291,-0.150624,-0.015320,0.010934,-0.093835,-0.016868,-0.163217,0.025731,0.139835
1,term,0.419421,1.000000,0.380916,0.222203,0.391490,0.037560,0.121298,0.168507,0.071892,...,-0.025081,-0.025631,-0.054567,-0.006950,-0.006271,-0.037208,-0.007022,-0.107663,0.027126,0.081283
2,int_rate,0.107936,0.380916,1.000000,0.125531,0.983716,-0.009644,-0.130340,0.247945,0.186759,...,0.015040,0.029464,0.060343,0.007946,0.048053,0.008425,0.006990,-0.220303,0.019134,0.205573
3,installment,0.968238,0.222203,0.125531,1.000000,0.123246,0.046088,0.447214,0.070460,0.058505,...,-0.075401,-0.066936,-0.142629,-0.014007,0.018273,-0.091803,-0.015263,-0.167306,0.021195,0.148928
4,sub_grade,0.110776,0.391490,0.983716,0.123246,1.000000,-0.009372,-0.131240,0.249515,0.193556,...,0.016111,0.030830,0.062274,0.008338,0.049477,0.009248,0.004894,-0.233935,0.033595,0.203965
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,vacation,-0.093835,-0.037208,0.008425,-0.091803,0.009248,0.010601,-0.012918,-0.003746,-0.004402,...,-0.009419,-0.007141,-0.021617,-0.002269,-0.008981,1.000000,-0.002912,0.003099,-0.001345,-0.001731
89,wedding,-0.016868,-0.007022,0.006990,-0.015263,0.004894,-0.016127,-0.005514,-0.006697,-0.016857,...,-0.003798,-0.002879,-0.008716,-0.000915,-0.003621,-0.002912,1.000000,0.005539,-0.008563,0.003553
90,Not Verified,-0.163217,-0.107663,-0.220303,-0.167306,-0.233935,-0.007304,-0.031259,-0.070663,-0.043369,...,0.003083,-0.021113,-0.012240,-0.007706,-0.032388,0.003099,0.005539,1.000000,-0.550008,-0.433339
91,Source Verified,0.025731,0.027126,0.019134,0.021195,0.033595,-0.067590,0.043428,0.013332,-0.072154,...,-0.000708,-0.003784,0.006555,-0.002530,0.010158,-0.001345,-0.008563,-0.550008,1.000000,-0.514331
